<a href="https://colab.research.google.com/github/YounSooKimTech/NLP_teamproject/blob/main/IMDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis

import numpy as np # linear algebra
import spacy
import string
import gensim
import operator
import re


In [ ]:
# get first page data
api_key = "50b3d92df451f49698a9387ad0edf464" # get TMDB API key 
# response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&language=en-US'+'&page='+str(1))


In [ ]:
films = response.json() 
films = films['results']
# create data frame for first page
films = pd.DataFrame(films)

In [ ]:
# Retriving data by looping through 500 pages 
for x in range (2,500,1):
  response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key +'&page='+str(x))
  re = response.json()
  results = re['results']
  df = pd.DataFrame(results)
  films = films.append(df)


In [ ]:
len(films)

9980

In [ ]:
films.head(10)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,text
0,False,/zt6sKnx9dEiRCb7oVMlfmmMGJMO.jpg,"[28, 35, 53]",718930,en,Bullet Train,Unlucky assassin Ladybug is determined to do h...,1339.013,/tVxDe01Zy3kZqaZRNiXFGDICdZk.jpg,2022-07-03,Bullet Train,False,7.5,2292,Bullet TrainUnlucky assassin Ladybug is determ...
1,False,/uSqLcj8V2qWYRbOXCfdwV90vASS.jpg,"[18, 28, 10752]",966220,uk,Снайпер. Білий ворон,Mykola is an eccentric pacifist who wants to b...,1318.138,/295gZzTXMvuiIG0U19h4M44PXxI.jpg,2022-05-03,Sniper: The White Raven,False,7.3,64,Sniper: The White RavenMykola is an eccentric ...
2,False,/tSxbUnrnWlR5dQvUgqMI7sACmFD.jpg,"[14, 28, 18]",779782,en,The School for Good and Evil,Best friends Sophie and Agatha navigate an enc...,1275.654,/6oZeEu1GDILdwezmZ5e2xWISf1C.jpg,2022-10-19,The School for Good and Evil,False,7.3,565,The School for Good and EvilBest friends Sophi...
3,False,/3PieOs1t6dPHWlgvX3XoqTIQLqN.jpg,"[12, 28, 878]",507086,en,Jurassic World Dominion,Four years after the destruction of Isla Nubla...,1267.666,/kAVRgw7GgK1CfYEJq8ME6EvRIgU.jpg,2022-06-01,Jurassic World Dominion,False,7.0,3827,Jurassic World DominionFour years after the de...
4,False,/nnUQqlVZeEGuCRx8SaoCU4XVHJN.jpg,"[14, 12, 10751]",532639,en,Pinocchio,A wooden puppet embarks on a thrilling adventu...,1266.466,/g8sclIV4gj1TZqUpnL82hKOTK3B.jpg,2022-09-07,Pinocchio,False,6.7,1057,PinocchioA wooden puppet embarks on a thrillin...
5,False,/olPXihyFeeNvnaD6IOBltgIV1FU.jpg,"[27, 9648, 53]",882598,en,Smile,"After witnessing a bizarre, traumatic incident...",1246.237,/aPqcQwu4VGEewPhagWNncDbJ9Xp.jpg,2022-09-23,Smile,False,6.7,495,"SmileAfter witnessing a bizarre, traumatic inc..."
6,False,/7ZO9yoEU2fAHKhmJWfAc2QIPWJg.jpg,"[53, 28, 878]",766507,en,Prey,"When danger threatens her camp, the fierce and...",1214.823,/ujr5pztc1oitbe7ViMUOilFaJ7s.jpg,2022-08-05,Prey,False,7.9,4581,"PreyWhen danger threatens her camp, the fierce..."
7,False,/2iGUavwv86Hubv3V5Iq4qEQXDfE.jpg,"[18, 53, 27]",848058,es,Cerdita,A bullied overweight teenager sees a glimpse o...,1201.565,/8EIQAvJjXdbNDMmBtHtgGqbc09V.jpg,2022-10-07,Piggy,False,6.7,188,PiggyA bullied overweight teenager sees a glim...
8,False,/mMA2YNddowV8MZtxpbn0a7Yilum.jpg,"[10752, 36, 28, 18]",928123,zh,长津湖之水门桥,"In the follow-up to ""The Battle At Lake Changj...",1184.147,/ugiL6wIhl1OfPyv1gqLkTe45jLl.jpg,2022-02-01,The Battle at Lake Changjin: Water Gate Bridge,False,6.4,24,The Battle at Lake Changjin: Water Gate Bridge...
9,False,/utUVo1wt3i4OdrQQS3I9QeegYsM.jpg,"[27, 53, 14]",780609,en,Men,"In the aftermath of a personal tragedy, Harper...",1169.442,/jo1Kv3P3UgDVk7JnUFr2Cl8WWUM.jpg,2022-05-20,Men,False,6.3,456,"MenIn the aftermath of a personal tragedy, Har..."


# Preprocessing

In [ ]:
films[["id","genre_ids", "title", "overview","adult"]]

,id,genre_ids,title,overview,adult
0,718930,"[28, 35, 53]",Bullet Train,Unlucky assassin Ladybug is determined to do h...,False
1,966220,"[18, 28, 10752]",Sniper: The White Raven,Mykola is an eccentric pacifist who wants to b...,False
2,779782,"[14, 28, 18]",The School for Good and Evil,Best friends Sophie and Agatha navigate an enc...,False
3,507086,"[12, 28, 878]",Jurassic World Dominion,Four years after the destruction of Isla Nubla...,False
4,532639,"[14, 12, 10751]",Pinocchio,A wooden puppet embarks on a thrilling adventu...,False
...,...,...,...,...,...
15,433627,"[53, 18, 80]",7 Days in Entebbe,"In 1976, four hijackers take over an Air Franc...",False
16,491926,"[10752, 36, 18]",Resistance,The story of mime Marcel Marceau as he works w...,False
17,33539,"[16, 10751, 12]",Once Upon a Forest,"A young mouse, mole and hedgehog risk their li...",False
18,44108,"[18, 10749, 27]",Nang Nak,"In a rural village in Thailand, Mak is sent to...",False


In [ ]:
# make a string

films["text"] = films["title"] + films["overview"]
films.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,text
0,False,/zt6sKnx9dEiRCb7oVMlfmmMGJMO.jpg,"[28, 35, 53]",718930,en,Bullet Train,Unlucky assassin Ladybug is determined to do h...,1339.013,/tVxDe01Zy3kZqaZRNiXFGDICdZk.jpg,2022-07-03,Bullet Train,False,7.5,2292,Bullet TrainUnlucky assassin Ladybug is determ...
1,False,/uSqLcj8V2qWYRbOXCfdwV90vASS.jpg,"[18, 28, 10752]",966220,uk,Снайпер. Білий ворон,Mykola is an eccentric pacifist who wants to b...,1318.138,/295gZzTXMvuiIG0U19h4M44PXxI.jpg,2022-05-03,Sniper: The White Raven,False,7.3,64,Sniper: The White RavenMykola is an eccentric ...
2,False,/tSxbUnrnWlR5dQvUgqMI7sACmFD.jpg,"[14, 28, 18]",779782,en,The School for Good and Evil,Best friends Sophie and Agatha navigate an enc...,1275.654,/6oZeEu1GDILdwezmZ5e2xWISf1C.jpg,2022-10-19,The School for Good and Evil,False,7.3,565,The School for Good and EvilBest friends Sophi...
3,False,/3PieOs1t6dPHWlgvX3XoqTIQLqN.jpg,"[12, 28, 878]",507086,en,Jurassic World Dominion,Four years after the destruction of Isla Nubla...,1267.666,/kAVRgw7GgK1CfYEJq8ME6EvRIgU.jpg,2022-06-01,Jurassic World Dominion,False,7.0,3827,Jurassic World DominionFour years after the de...
4,False,/nnUQqlVZeEGuCRx8SaoCU4XVHJN.jpg,"[14, 12, 10751]",532639,en,Pinocchio,A wooden puppet embarks on a thrilling adventu...,1266.466,/g8sclIV4gj1TZqUpnL82hKOTK3B.jpg,2022-09-07,Pinocchio,False,6.7,1057,PinocchioA wooden puppet embarks on a thrillin...


In [ ]:
df = films[["id","genre_ids", "title", "overview","adult"]]
df.head(10)

,id,genre_ids,title,overview,adult
0,718930,"[28, 35, 53]",Bullet Train,Unlucky assassin Ladybug is determined to do h...,False
1,966220,"[18, 28, 10752]",Sniper: The White Raven,Mykola is an eccentric pacifist who wants to b...,False
2,779782,"[14, 28, 18]",The School for Good and Evil,Best friends Sophie and Agatha navigate an enc...,False
3,507086,"[12, 28, 878]",Jurassic World Dominion,Four years after the destruction of Isla Nubla...,False
4,532639,"[14, 12, 10751]",Pinocchio,A wooden puppet embarks on a thrilling adventu...,False
5,882598,"[27, 9648, 53]",Smile,"After witnessing a bizarre, traumatic incident...",False
6,766507,"[53, 28, 878]",Prey,"When danger threatens her camp, the fierce and...",False
7,848058,"[18, 53, 27]",Piggy,A bullied overweight teenager sees a glimpse o...,False
8,928123,"[10752, 36, 28, 18]",The Battle at Lake Changjin: Water Gate Bridge,"In the follow-up to ""The Battle At Lake Changj...",False
9,780609,"[27, 53, 14]",Men,"In the aftermath of a personal tragedy, Harper...",False


In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)

    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [ ]:
print ('Cleaning and Tokenizing...')
%time df['overview_tokenized'] = df['overview'].map(lambda x: spacy_tokenizer(x))

df.head()

Cleaning and Tokenizing...
CPU times: user 1min 37s, sys: 412 ms, total: 1min 38s
Wall time: 1min 38s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,genre_ids,title,overview,adult,overview_tokenized
0,718930,"[28, 35, 53]",Bullet Train,Unlucky assassin Ladybug is determined to do h...,False,"[unlucky, assassin, ladybug, determined, job, ..."
1,966220,"[18, 28, 10752]",Sniper: The White Raven,Mykola is an eccentric pacifist who wants to b...,False,"[mykola, eccentric, pacifist, want, useful, hu..."
2,779782,"[14, 28, 18]",The School for Good and Evil,Best friends Sophie and Agatha navigate an enc...,False,"[good, friend, sophie, agatha, navigate, encha..."
3,507086,"[12, 28, 878]",Jurassic World Dominion,Four years after the destruction of Isla Nubla...,False,"[year, destruction, isla, nublar, biosyn, oper..."
4,532639,"[14, 12, 10751]",Pinocchio,A wooden puppet embarks on a thrilling adventu...,False,"[wooden, puppet, embark, thrilling, adventure,..."


In [ ]:
movie_plot = df['overview_tokenized']
movie_plot[0:5]

0    [unlucky, assassin, ladybug, determined, job, ...
1    [mykola, eccentric, pacifist, want, useful, hu...
2    [good, friend, sophie, agatha, navigate, encha...
3    [year, destruction, isla, nublar, biosyn, oper...
4    [wooden, puppet, embark, thrilling, adventure,...
Name: overview_tokenized, dtype: object